# Create a multi span reinforced concrete bridge deck with pier columns</br>
(v21.1 and later)

<i>Please note this is an example of how to use the LPI functionality to create a bridge model demonstrating the use of multiple load types. <br/>It is not an example of how to model such structures</i>

In [ ]:
solve_analyses = False
dmi_use_grid = True
influence_envelope = False # Otherwise individual assignmemnts
N_NODE = 3  # For non enveloped influnece analysis, add influence assignments and avery N nodes

Define model parameters

In [ ]:
# Geometric
deck_width = 11
deck_overhang = 1.5
deck_thk = 1.0
deck_thk_outstand = 0.25
deck_thk_pier = 1.4
span_lengths = [15, 20, 15]
column_dia = 1.1
pier_head = 2.0
abut_width = 1.0
pier_height = 5
skew_angle = 25

# Meshing
mesh_size = 2

# Loading area
carriageway_width = 11

# Loads
surf_load_intensity = -2.5
parapet_load_udl = -2.5
footpath_surface_load_intensity = -1 # Additional thickness of footpath
settlement = -0.01 # m

# Temperatures
temperature_increase = 20
temperature_decrease = -20
temp_max = 25
temp_min = 0.0

# Live loads
pedestrian_live_load = -5 # kpa
breaking_load = 900 # kN
horiz_load = 200    # kN
pier_impact_load = 1000 # kN
deck_impact_load = 1000 # kN/m

In [ ]:
# Calculated dimensions
overall_length = sum(span_lengths) + abut_width
overall_width  = 2*deck_overhang + deck_width

footway_width = (overall_width-carriageway_width)/2

breaking_load_intensity = breaking_load / (carriageway_width*overall_length)

eccentcity_pier = (deck_thk_pier-deck_thk)/2
eccentcity_outstand = (deck_thk_outstand-deck_thk)/2

Connect to LUSAS and either create a new model or delete the contents of the existing one - be careful if you have a model already loaded

In [ ]:
import numpy as np
import math
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0") # Connect to LUSAS Modeller
import sys; sys.path.append('../') # Locate the helpers file
from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

# If there is not currently a model open, create one
if not lusas.existsDatabase():
    lusas.newProject("Structural", "Concrete Slab Bridge Deck")
else:
    Helpers.reset_database_contents(lusas.database())

db = lusas.database() # Reference to the model database for convenience

# 3D model with Z vertical
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")

# Set the unit system
db.setModelUnits("kN,m,t,s,C")

Create the attributes to be assigned to the geometric features. Mesh, Geometric and Material.

In [ ]:
# Beam element mesh attribute for the pier columns
beam_mesh_attr = db.createMeshLine("LMsh2").setSize("BMI21", mesh_size)
# Set the upper section which overlaps with the deck to be rigid
#beam_mesh_attr.setEndRigidZoneSameAsStart(False).setRigid("Start", "length", False, 1.0, 1.0)

# Pier column geometric attribute
pier_geom_attr = Helpers.create_circular_section(db, 'Pier Column', column_dia)

# Concrete material attribute
concrete_mat_attr = db.createIsotropicMaterial("Concrete", 34.8E6, 0.2, 2.4)
concrete_mat_attr.setValue("alpha", 10e-6) # Coefficient of thermal expansion
# Concrete material assigned to all
db.setAsDefault("Material", concrete_mat_attr)

# Deck mesh attribute, thick shell elements
deck_mesh_attr    = db.createMeshSurface("SMsh1").setRegularSize("QTS4", mesh_size, True)
# Deck surface thickness attribute
deck_surface_attr      = db.createGeometricSurface("Deck Slab").setSurface(deck_thk, 0.0)
pier_head_surface_attr = db.createGeometricSurface("Deck Slab at Pier").setSurface(deck_thk_pier, eccentcity_pier)
outstand_surface_attr  = db.createGeometricSurface("Deck Outstand").setSurface(deck_thk_outstand, eccentcity_outstand)

# Support attributes
support_pinned_attr = db.createSupportStructural("Pinned").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F") # F=Free, R=Restrained
support_slide_attr  = db.createSupportStructural("Slide").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")
support_fixed_attr  = db.createSupportStructural("Fully Fixed").setStructural("R", "R", "R", "R", "R", "R", "F", "F", "C", "F")

Define some useful helper functions for creating lines and points. This keeps the code clean and avoids duplication

In [ ]:
# Define a useful helper function to create a line from two point coordinates
def create_line(p1:list, p2:list) -> 'IFLine':
    # geometryData object contains all the settings to perform a geometry creation
    geom_data = lusas.geometryData().setAllDefaults()  
    # set the options for creating straight lines from coordinates
    geom_data.setCreateMethod("straight")        
    geom_data.setLowerOrderGeometryType("coordinates")        
    # Add the cordinates, lines directions will follow the order of the coordinates
    geom_data.addCoords(p1[0], p1[1], p1[2])    # Set the coordinates of the first point X,Y,Z
    geom_data.addCoords(p2[0], p2[1], p2[2])    # Set the coordinates of the second point X,Y,Z
    # Create the line, get the line object from the returned object set and return the 1 and only line
    return win32.CastTo(db.createLine(geom_data).getObject("Line"), "IFLine")
    
# Define a useful helper function to create a line from two point objects
# Note that we expect two IFPoint objects, these are references to points already created in the model
def create_line_from_points(p1:'IFPoint', p2:'IFPoint') -> 'IFLine':
    # geometryData object contains all the settings to perform a geometry creation
    geom_data = lusas.geometryData().setAllDefaults()         
    # set the options for creating straight lines from points
    geom_data.setCreateMethod("straight")        
    geom_data.setLowerOrderGeometryType("points")
    # Create an object set to contain the points and use this set to create the line
    obs = lusas.newObjectSet()                 
    obs.add(p1)
    obs.add(p2)
    # Create the line, get the line object from the returned object set and return the 1 and only line
    return win32.CastTo(obs.createLine(geom_data).getObject("Line"), "IFLine")


# Define a useful helper function to create a point from coordinates
def create_point(x:float, y:float, z:float) -> 'IFPoint':
    # geometryData object contains all the settings to perform a geometry creation
    geom_data = lusas.geometryData().setAllDefaults() 
    # set the options for creating points from coordinates
    geom_data.setLowerOrderGeometryType("coordinates")
    # Add the coordinates
    geom_data.addCoords(x, y, z)
    # Create the point and return it. 
    # Note that createPoint returns and IFObjectSet from which we can get the point. (this allows multiple points to be created at once, here we only have 1)
    return win32.CastTo(db.createPoint(geom_data).getObject("Point"), "IFPoint")

In [ ]:
def create_surface_from_points(points:list):
    # geometryData object contains all the settings to perform a geometry creation
    geom_data = lusas.geometryData().setAllDefaults() 
    # set the options for creating points from coordinates
    geom_data.setLowerOrderGeometryType("points")
    # Create an object set to contain the points and use this set to create the line
    obs = lusas.newObjectSet()
    for p in points:
        obs.add(p)             

    # Create the Surface, get the surface object from the returned object set and return the 1 and only surface
    return win32.CastTo(obs.createSurface(geom_data).getObject("Surface"), "IFSurface")

In [ ]:
# Helper function to calculate beam coordinates accounting for the skew angle
def skew_x(y:float)->float:
    return y * math.tan(math.radians(skew_angle))

## Create the model geometry

In [ ]:
# Create a list of x coordinates at support locations
support_xs = [abut_width/2] + list(np.add.accumulate(span_lengths)+abut_width/2)

# X Coordinates for the entire deck grid
x_grid = list()
x = abut_width/2
# First support
x_grid.extend([0.0, x, x+abut_width/2])
# Pier lines
for i in range(0, len(span_lengths)-1):
    x+=span_lengths[i]
    x_grid.extend([x-pier_head/2, x, x+pier_head/2])
# final support
x+=span_lengths[-1]
x_grid.extend([x-abut_width/2, x, x+abut_width/2])

# Y Coordinates for deck grid
y_grid = [-deck_width/2-deck_overhang, -deck_width/2, -deck_width/3, 0.0, deck_width/3, deck_width/2, deck_width/2+deck_overhang]

In [ ]:
# Create the actual points in the model [Y][X]
deck_points = [[create_point(x+skew_x(y), y, 0.0) for x in x_grid] for y in y_grid]

In [ ]:
''' Create Deck surfaces '''
# Set the default mesh attribute which is automatically assigned to each newly created surface
db.setAsDefault("Surface Mesh", deck_mesh_attr)

# Create the surfaces for the first outstand
db.setAsDefault("Surface Geometric", outstand_surface_attr)
for ix in range(0, len(x_grid)-1):
    create_surface_from_points([ deck_points[0][ix], deck_points[0][ix+1], deck_points[1][ix+1], deck_points[1][ix] ])

# Create the surfaces for the main deck
db.setAsDefault("Surface Geometric", deck_surface_attr)
for iy in range(1, len(y_grid)-2):
    for ix in range(0, len(x_grid)-1):
        create_surface_from_points([ deck_points[iy][ix], deck_points[iy][ix+1], deck_points[iy+1][ix+1], deck_points[iy+1][ix] ])

# Create the surfaces for the second outstand
db.setAsDefault("Surface Geometric", outstand_surface_attr)
for ix in range(0, len(x_grid)-1):
    create_surface_from_points([ deck_points[-2][ix], deck_points[-2][ix+1], deck_points[-1][ix+1], deck_points[-1][ix] ])


In [ ]:
# Modify the geometric assignment for the deck over the supports

# Determine the array indicies for all support points
support_indices = [1]
for s in span_lengths:
    support_indices.append(support_indices[-1]+3)

# For each point in the support line, get the connected surfaces and assign the geometric attribute
for ix in support_indices:
    objs = lusas.newObjectSet()
    for iy in range(2, len(y_grid)-2): # Ignore the outstands
        objs.add(deck_points[iy][ix])
    objs.addHOF("Surface")
    objs.remove("Point")
    pier_head_surface_attr.assignTo(objs)

In [ ]:
''' Pier columns '''
# Set the default mesh and geometric section attribute to be automatically assigned to the newly created pier lines
db.setAsDefault("Mesh", beam_mesh_attr)
db.setAsDefault("Line Geometric", pier_geom_attr)

# List of List of supported points, we'll use this later to assign displacements in the settlement analysis
all_support_points = list()
# List pf pier lines, we'll use this later to assign impact loads
pier_column_lines = list()

# Pier indicies
pier_indices_X = support_indices[1:-1]
pier_indices_Y = [2,3,4]

# Create pier lines
for ix in pier_indices_X:
    supports = list()
    for iy in pier_indices_Y:

        deck_point = deck_points[iy][ix]
        ground_point = create_point(deck_point.getX(), deck_point.getY(), -pier_height)
        pier_column_lines.append(create_line_from_points(deck_point, ground_point))

        # Assign the fixed support attribute to the bottom point
        support_fixed_attr.assignTo(ground_point)
        supports.append(ground_point)

    all_support_points.append(supports)


# End 1 pinned supports
all_support_points.insert(0, list())
for iy in pier_indices_Y:
    deck_point = deck_points[iy][1]
    support_pinned_attr.assignTo(deck_point)
    all_support_points[0].append(deck_point)

# End 2 slide supports
all_support_points.append(list())
for iy in pier_indices_Y:
    deck_point = deck_points[iy][len(x_grid) - 2]
    support_slide_attr.assignTo(deck_point)
    all_support_points[-1].append(deck_point)

In [ ]:
# Turn off default attribute assignments
db.setAsDefault("Mesh", "None")
db.setAsDefault("Surface Mesh", "None")
db.setAsDefault("Line Geometric", "None")
db.setAsDefault("Surface Geometric", "None")
db.setAsDefault("Isotropic Material", "None")

Generate the mesh

In [ ]:
db.resetMesh()
db.updateMesh()

Define the carriageway by two kerb lines

In [ ]:
overall_length = sum(span_lengths) + abut_width
# Create lines to represent the carriageway kerbs that can be used for the Vehicle Load Optimiser
curb_line1 = create_line([-10,  carriageway_width/2, 0.5], [10 + overall_length,  carriageway_width/2, 0.5])
curb_line2 = create_line([-10, -carriageway_width/2, 0.5], [10 + overall_length, -carriageway_width/2, 0.5])
# Set the colour to distinguish them from model lines
curb_line1.setPen(2)
curb_line2.setPen(2)

# Loads

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
# NOTE: getLoadset and setName function returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase").addGravity(True)

In [ ]:
# Create a loadcase for the Parapet loads to be applied to the entire deck surface
parapet_loadcase = db.createLoadcase("Parapet Loads")
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(parapet_loadcase)
# Create a locally distributed load for Parapet loads
parapet_load_attr = db.createLoadingGlobalDistributed("Parapet Loads").setGlobalDistributed("Length", 0.0, 0.0, parapet_load_udl, 0.0, 0.0, 0.0)
# and assign to all the edge lines

# Start with the first point at either edge of the deck
for i in [0, len(y_grid)-1]:
    p0 = deck_points[i][0]
    p1 = deck_points[i][1]
    # Find the line that connects the first and second points, i.e. along the x axis and add to the object set
    objs = lusas.newObjectSet()
    for geom in p0.getHOFs():
        if geom.getEndPoint() == p1 or geom.getStartPoint() == p1: # Line directions are different so check both points
            objs.add(geom)
    # Get the object set to include all connected colinear lines
    objs.addColinearNeighbours(1)
    # And assign the parapet loads
    parapet_load_attr.assignTo(objs, assignment)

In [ ]:
# Create a loadcase for the Surfacing loads to be applied to the entire deck surface
surfacing_loadcase = db.createLoadcase("Surfacing")
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults().setLoadset(surfacing_loadcase)
# Create a locally distributed load for Surfacing loads
surf_load_attr = db.createLoadingLocalDistributed("Surfacing").setLocalDistributed(0.0, 0.0, surf_load_intensity, "surface")
# and assign to all the surfaces
surf_load_attr.assignTo("Surfaces", assignment)

Discrete loads can be be re-used in the model and assigned to a different origin. We'll do that for the footpath loads.<br/>
We'll make a generic function to create a 4 noded discrete patch load the size of the footpath.

In [ ]:
POINT_OFFSET = 9 # Points created for assignment of discrete loads will be offset from the deck for clarity
Z_OFFSET = 1

def create_footpath_load(name:str, load1:float, load2:float)->'IFLoadingDiscretePatch':
    attr = db.createLoadingDiscretePatch(name).setDiscretePatch("surf4", "Z")
    # Add the points of an 4 node patch load in which the load intensity varies across the width
    # The x ccordinates are increased by 1 because we'll assign these loads to points created at x=-1 to hold these loads
    attr.addRow(POINT_OFFSET,                0.0,           Z_OFFSET, load1)
    attr.addRow(POINT_OFFSET,                footway_width, Z_OFFSET, load2)
    attr.addRow(POINT_OFFSET+overall_length, footway_width, Z_OFFSET, load2)
    attr.addRow(POINT_OFFSET+overall_length, 0.0,           Z_OFFSET, load1)
    # Explicitly specify how the patch load should be divided up.
    # We know the width of the footway is small so we dont need so many divisions
    # Limiting the divisions speeds up the time to resolve the patch load into nodal loads
    attr.setDivisions(math.ceil(footway_width), 40)

    return attr

In [ ]:
# Create a surfacing load using the generic fucntion
footpath_surf_load_attr = create_footpath_load("Footpath Surface", footpath_surface_load_intensity, footpath_surface_load_intensity)

# Assign this to a new point using the assignment object set up for the deck surfacing loads
footpath_point1 = create_point(-POINT_OFFSET, -overall_width/2, 0.0)
footpath_point1.setPen(2)
footpath_surf_load_attr.assignTo(footpath_point1, assignment)
# And again to a 2nd new point
footpath_point2 = create_point(-POINT_OFFSET, carriageway_width/2, 0.0)
footpath_point2.setPen(2)
footpath_surf_load_attr.assignTo(footpath_point2, assignment)

# Note that we set the colour of the new points to blue so we can identify that they are not part of the model and only used to locate the discrete loads

Create a settlement load acting only on each support

In [ ]:
# Create the prescribed displacement attribute to represent vertical settlement
settlement_attr = db.createPrescribedDisplacementLoad("Settlement", "Total")
settlement_attr.setDisplacement("W", settlement)

# Create a loadcase for each support settlement
for i in range(0, len(all_support_points)):
    settlement_loadcase = db.createLoadcase(f"Settlement {i+1}")
    # Set the loadcase in the assignment object
    assignment = lusas.assignment().setAllDefaults().setLoadset(settlement_loadcase)
    # Use the surface set defined early to assign the load to all deck surfaces
    for p in all_support_points[i]:
        settlement_attr.assignTo(p, assignment)

# Variable Loads

Create temperature loads

In [ ]:
# Create a loadcases for temeperature
temp_warm_loadcase = db.createLoadcase("Temperature +dT")
temp_cool_loadcase = db.createLoadcase("Temperature -dT")
# Temperature loading warming 
temp_warm_attr = db.createLoadingTemperature("Warming").setTemperature("element", temperature_increase, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Temperature loading cooling 
temp_cool_attr = db.createLoadingTemperature("Cooling").setTemperature("element", temperature_decrease, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_warm_loadcase)
temp_warm_attr.assignTo("Surfaces", assignment)
# Assign the temperature cooling load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_cool_loadcase)
temp_cool_attr.assignTo("Surfaces", assignment)

In [ ]:
# Create a loadcases for temeperature Gradient
temp_warm_grad_loadcase = db.createLoadcase("Temperature +dT Grad")
temp_cool_grad_loadcase = db.createLoadcase("Temperature -dT Grad")
# Temperature loading warming 
temp_warm_attr = db.createLoadingTemperatureProfile("Warming Gradient").setTopTemperature(temp_max).setBottomTemperature(temp_min)
# Temperature loading cooling 
temp_cool_attr = db.createLoadingTemperatureProfile("Cooling Gradient").setTopTemperature(temp_min).setBottomTemperature(temp_max)
# Assign the temperature warming load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_warm_grad_loadcase)
temp_warm_attr.assignTo("Surfaces", assignment)
# Assign the temperature cooling load
assignment = lusas.assignment().setAllDefaults().setLoadset(temp_cool_grad_loadcase)
temp_cool_attr.assignTo("Surfaces", assignment)

Pedestrian live loads

In [ ]:
# We can reuse the discrete patch load fucntion to create pedestrian live loads 
ped_live_load_attr = create_footpath_load("Pedestrian Live load", pedestrian_live_load, pedestrian_live_load)

# Create a loadcase for the pedestrian live loads
ped_loads_loadcase = db.createLoadcase("Pedestrian live loads")

# Redefine the assignment object for the discrete load assignment as we did for the surfacing loads
assignment = lusas.assignment().setAllDefaults().setLoadset(ped_loads_loadcase)
assignment.setLoadMoving("Exclude All Load")
assignment.setSearchAssignType("area")

# Again, assign the same load to the two dedicated points
ped_live_load_attr.assignTo(footpath_point1, assignment)
ped_live_load_attr.assignTo(footpath_point2, assignment)

Breaking load

In [ ]:

# Create a discrete patch load with the loads acting in the X direction.
# Patch loads are projected onto the structure in the direction of their normal. Which will be the Z axis in this case.
breaking_load_attr = db.createLoadingDiscretePatch("Breaking load 1").setDiscretePatch("surf4", "X")
# Add the points of an 4 node patch load
breaking_load_attr.addRow(POINT_OFFSET,                -carriageway_width/2, Z_OFFSET, breaking_load_intensity)
breaking_load_attr.addRow(POINT_OFFSET,                 carriageway_width/2, Z_OFFSET, breaking_load_intensity)
breaking_load_attr.addRow(POINT_OFFSET+overall_length,  carriageway_width/2, Z_OFFSET, breaking_load_intensity)
breaking_load_attr.addRow(POINT_OFFSET+overall_length, -carriageway_width/2, Z_OFFSET, breaking_load_intensity)

# Create a loadcase for the braking loads
breaking_loads_loadcase1 = db.createLoadcase("Breaking load +ve")
breaking_loads_loadcase2 = db.createLoadcase("Breaking load -ve")

# Redefine the assignment object for the discrete load assignment as we did for the surfacing loads
assignment = lusas.assignment().setAllDefaults().setLoadset(breaking_loads_loadcase1)
assignment.setLoadMoving("Exclude All Load")
assignment.setSearchAssignType("area")

# Assign to the point at the origin
point0 = create_point(-POINT_OFFSET, 0.0, 0.0)
point0.setPen(2)
breaking_load_attr.assignTo(point0, assignment)

# Assign the same load with a negative load factor for the opposite direction
assignment.setLoadFactor(-1)
assignment.setLoadset(breaking_loads_loadcase2)
breaking_load_attr.assignTo(point0, assignment)

Horizontal loads

In [ ]:
# Create the concentrated load attribute
horiz_load_pos_attr = db.createLoadingConcentrated("Horizontal +ve")
horiz_load_pos_attr.setConcentrated(0.0, horiz_load, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
horiz_load_neg_attr = db.createLoadingConcentrated("Horizontal -ve")
horiz_load_neg_attr.setConcentrated(0.0, -horiz_load, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

# Assign positive and negative horizontal loads to each pier
count = 1
for ix in pier_indices_X:
    horiz_pos_loadcase = db.createLoadcase(f"Horizontal +ve {count}")
    horiz_neg_loadcase = db.createLoadcase(f"Horizontal -ve {count}")
    # Assign the lateral load to the central column of each pier
    deck_point = deck_points[pier_indices_Y[1]][ix]
    assignment = lusas.assignment().setAllDefaults().setLoadset(horiz_pos_loadcase)
    horiz_load_pos_attr.assignTo(deck_point, assignment)
    assignment = lusas.assignment().setAllDefaults().setLoadset(horiz_neg_loadcase)
    horiz_load_neg_attr.assignTo(deck_point, assignment)
    count+=1

# Impact Loads

In [ ]:
# Impact load defined as a point load at mid height of the pier leg
impact_load_attr = db.createLoadingBeamPoint("Pier Impact").setBeamPoint("Parametric", "Global", "beam")
impact_load_attr.addRow(0.5, -pier_impact_load/2, pier_impact_load, 0.0, 0.0, 0.0, 0.0)

# Create the loadcase - we'll just consider a single case here
impact_loadcase = db.createLoadcase(f"Pier Impact")
# Assign to the first pier leg
impact_load_attr.assignTo(pier_column_lines[0], assignment.setAllDefaults().setLoadset(impact_loadcase))

In [ ]:
# Impact load applied to the deck

# Create a discrete line load acting in the global Y direction, but projected onto the model in the Z direction
deck_impact_load_attr = db.createLoadingDiscretePatch("Deck Impact").setDiscretePatch("line2", "Y", [0,0,1])
# Specify the load position relative to the 
deck_impact_load_attr.addRow(span_lengths[1]/2 -0.5, 0.0, 0.0, deck_impact_load)
deck_impact_load_attr.addRow(span_lengths[1]/2 +0.5, 0.0, 0.0, deck_impact_load)

# Create the loadcase
impact_loadcase = db.createLoadcase(f"Deck Impact")
# Assign to the point at the start of span 2
p0 = deck_points[1][4]

deck_impact_load_attr.assignTo(p0, assignment.setAllDefaults().setLoadset(impact_loadcase))


# Load Groups

In [ ]:
''' Create some intermediate load groups to help with later design combinations '''

# Get all the loadcases in the model
loadcases = db.getLoadsets("Loadcase")

# Envelope Settlement cases
settlement_env = db.createEnvelope("Settlement Env")
for lc in loadcases:
    if "Settlement" in lc.getName():
        settlement_env.addEntry(lc)

# Envelope +/- Temperature cases
temp_annual_env = db.createEnvelope("Temp Annual Env")
temp_annual_env.addEntry(temp_warm_loadcase)
temp_annual_env.addEntry(temp_cool_loadcase)

temp_daily_env = db.createEnvelope("Temp Daily Env")
temp_daily_env.addEntry(temp_warm_grad_loadcase)
temp_daily_env.addEntry(temp_cool_grad_loadcase)

# Then combine seasonal and daily variations
temp_comb = db.createCombinationSmart("Temperature")
temp_comb.addEntry(1,0,temp_annual_env)
temp_comb.addEntry(1,0,temp_daily_env)

# Envelope Braking Loadcases
braking_env = db.createEnvelope("Braking Env")
for lc in loadcases:
    if "Braking" in lc.getName():
        braking_env.addEntry(lc)

# Envelope Horizontal Loadcases
braking_env = db.createEnvelope("Horizontal Env")
for lc in loadcases:
    if "Horizontal" in lc.getName():
        braking_env.addEntry(lc)

# Influence analysis

In [ ]:
# An influence analysis considers unit loads applied to a group of elements in order to generate influence surfaces.
# We need to specify the group of elements by assigning a search area
# Create a search area attribute
search_area_attr = db.createSearchArea("Deck Slab")
# Assign the search area atttribute to the same surfaces
search_area_attr.assignTo("Surfaces", lusas.assignment().setAllDefaults())

In [ ]:
# Create a Direct Method Influence Analysis
dmi_analysis = db.createAnalysisDirectMethodInf("DMI")
# Specify the elements in the search area deck slab
dmi_analysis.setSearchArea("Deck Slab")
dmi_analysis.setSearchAssignType("area")
dmi_analysis.setIsGrillageType(1)
dmi_analysis.setGridCentreline("X")
dmi_analysis.setGridWidth(carriageway_width)
dmi_analysis.setIncludeMidSideNodes(False)
dmi_analysis.setSelectedResultsGroup("assignments")
 # Use Grid
if dmi_use_grid:
    dmi_analysis.setDoMatchNodes(False)
    # Generate the DMI elements for the grid
    dmi_analysis.createElements()

In [ ]:
if influence_envelope:
    influ_env_attr_My = db.createInfluenceEnvelope("Deck Major Bending Mx Env")
    influ_env_attr_My.setResultsTransformGlobal()
    influ_env_attr_My.setEntity("Force/Moment - Thick Shell")
    influ_env_attr_My.addComponentNames(["Mx"])
    influ_env_attr_My.includeCoincident(True)

    # Assign the influence attribute to all the surfaces in the DMI analysis
    assignment = lusas.assignment().setAllDefaults().setLoadsetOff().setAnalysis("DMI")
    influ_env_attr_My.assignTo("Surfaces", assignment)

    # Separate assignment for the column elements - Note that these envelopes cannot be combined together as they optimise different entities
    influ_env_attr_cols = db.createInfluenceEnvelope("Column All")
    influ_env_attr_cols.setResultsTransformGlobal()
    influ_env_attr_cols.setEntity("Force/Moment - Thick 3D Beam")
    influ_env_attr_cols.addComponentNames(["Fx", "Fy", "Fz", "Mx", "My", "Mz"])
    influ_env_attr_cols.includeCoincident(True)

    # Assign the influence attribute to all the lines with a piers in the DMI analysis
    influ_env_attr_cols.assignTo(lusas.newObjectSet().add(pier_geom_attr), assignment)

In [ ]:
if not influence_envelope:
    # Create an influnec attribute which is assigned to speciifc locations in the model.
    inf_attr = db.createDirectInfluence("Deck Major Bending Mx")
    inf_attr.setResultsTransformGlobal()
    inf_attr.setInfluence("Force/Moment - Thick Shell", "Mx")
    
    # For longitudinal lines at the following indicies,
    # assign the influence attribute at every N nodes along the line
    # Note the order of the nodes follows the direction of the lines
    for i in [2,3,4]:
        objs = lusas.newObjectSet()
        p0 = deck_points[i][0]
        p1 = deck_points[i][1]
        for hof in p0.getHOFs():
            if hof.getStartPoint() == p1 or hof.getEndPoint() == p1:
                objs.add(hof)
                objs.addColinearNeighbours(1)

        lines = objs.getObjects("Lines")
        objs.remove("all")

        # Skip the end two lines which are the rigid end beam
        for line in lines[2:-2]:
            nodes = line.getNodes()
            for n, node in enumerate(nodes):
                if n%N_NODE == 0:
                    objs.add(node)

        # Set the assignment data to assign the influence attributes to nodes in the DMI Analysis
        assignment = lusas.assignment().setAllDefaults().setAnalysis("DMI").addToSelection("Node")
        inf_attr.assignTo(objs, assignment)

Solve the analyses

In [ ]:
# Solve each analysis and open all the available results files
if solve_analyses:
    db.getAnalysis("Analysis 1").solve(True)
    db.openAllResults(False)

In [ ]:
lusas.view().insertValuesLayer()
lusas.view().values().setResultsTransformNone()
lusas.view().values().setResults("Reaction", "FZ")

Set the view to be perspective mode

In [ ]:
lusas.view().setProjection(1)
lusas.view().setIsometric()
lusas.view().setScaledToFit(True)

Set the view to display the assigned loading in the attributes layers as well as the geometric "fleshing"

In [ ]:
lusas.view().insertAttributesLayer()
lusas.view().attributes().drawPatchByDefinition(True)
lusas.view().attributes().visualiseAll("Loading")
lusas.view().attributes().visualiseAll("Geometric")
#lusas.view().attributes().visualiseAllTransparent("Geometric")
